<a href="https://colab.research.google.com/github/napoles-uach/ML-FCQ/blob/main/MaterialDidacticoUDA_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MATERIAL DIDÁCTICO INNOVADOR. Para uso en la UDA "*Introducción al Machine Learning*".
## Elaborado por los profesores **José Manuel Nápoles Duarte** y **Juan Pedro Palomares Baez**.

## Este material es una guia para diseñar una web app para reconocimiento de imagenes. Se utiliza la plataforma [Teachable Machine](https://teachablemachine.withgoogle.com/train) para generar un modelo de red neuronal convolucional.




# 1) Instalación de Liberías requeridas:
* [Keras](https://keras.io/)
* [Pillow](https://python-pillow.org/)
* [Numpy](https://numpy.org/)
* [Streamlit](https://streamlit.io/)
* [pyngrok](https://pypi.org/project/pyngrok/)

## Se instalan las librerías mediante el comando `pip`.

In [ ]:
!pip install keras pillow numpy
!pip -q install streamlit
!pip -q install pyngrok

# 2) A continuación se descarga con el comando `wget` el modelo entrenado que fue previamente generado en Teachable Machine y que es guardado en el archivo `keras_model.h5`. 

In [ ]:
!wget https://github.com/napoles-uach/streamlit_apps/raw/main/Streamlit_Colab/Models/Dog_Cat.h5
!mv Dog_Cat.h5 keras_model.h5

# 3) A continuación se genera el archivo `app.py` el cual contiene el código python para generar la web app.

In [ ]:
%%writefile app.py
import streamlit as st
from img_classification import teachable_machine_classification
import keras
from PIL import Image, ImageOps
import numpy as np
st.title("Clasificación de Imagenes con Teachable Machine")
st.header("Ejemplo para clasificación de perros y gatos")
st.write("Sube una imagen de un perro o un gato para que el modelo haga la clasificación")

uploaded_file = st.file_uploader("Escoge una imagen en formato jpg", type="jpg")
if uploaded_file is not None:
  image = Image.open(uploaded_file)
  st.image(image, caption='Imagen subida', use_column_width=True)
  st.write("")
  st.write("Clasificando, espera un momento...")
  label = teachable_machine_classification(image, 'keras_model.h5') # Name of the model from Teachablemachine
  if label == 0:
    st.write("Es un perro :dog: :+1:")
  else:
    st.write("Es un gato :cat: :+1:")

# 4) La siguiente celda es el código en el que se crea la función para llamar al modelo. **Esta función no debe modificarse.**




In [ ]:
%%writefile img_classification.py
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np


def teachable_machine_classification(img, weights_file):
    # Load the model
    model = tensorflow.keras.models.load_model(weights_file)

    # Create the array of the right shape to feed into the keras model
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    image = img
    #image sizing
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.ANTIALIAS)

    #turn the image into a numpy array
    image_array = np.asarray(image)
    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

    # Load the image into the array
    data[0] = normalized_image_array  # (Not sure if this is needed, but gives an error!!!)

    # run the inference
    prediction = model.predict(data)
    return np.argmax(prediction) # return position of the highest probability

# 5) La siguiente celda debe mantenerse funcionando para generar la liga a la applicación web. Al detenerse, el link quedará roto.

In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(port='80')
print('Link to web app:')
print (public_url)
!streamlit run --server.port 80 app.py >/dev/null